In [1]:
import numpy as np
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision.utils import save_image
from glob import glob

In [19]:
class VoiceMFCC(Dataset):
    """Voice MFCC spectra dataset."""

    def __init__(self, csv_files, standardize=False):
        """
        Preconditions: csv files must contain matrices of the same dimension
        Args:
            csv_files (string or list): list of filenames/pathnames of csv files 
                                        with MFCC spectrogram matrices
            transform (callable, optional): Optional transform to be applied
                                            on a sample.
        """
        # ensure csv_files is a list
        if type(csv_files) == str:
            csv_files = [csv_files]
        
        # load csv files with the MFCC spectra into a 3D tensor
        matrices = []
        for f in csv_files:
            matrix = np.loadtxt(f, delimiter=',', dtype=np.float32)
            if standardize:
                matrix = (matrix - np.mean(matrix)) / np.std(matrix)
            matrices.append(matrix)
        self.X = torch.Tensor(matrices)
        N, D, M = self.X.shape
        self.X = self.X.view(N, 1, D, M) # THIS LINE DIDNT SEEM TO FIX OUR PROBLEM...
        
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        sample = self.X[idx]

        return sample 

In [20]:
# import dataset
csv_files = glob("data/train_mfcc/*.csv")
dataset = VoiceMFCC(csv_files, standardize=True)

In [21]:
# split dataset
TRAIN_SIZE = 9099
VALID_SIZE = 2000
TEST_SIZE = 2000
train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, 
                                                                                [TRAIN_SIZE, 
                                                                                 VALID_SIZE, 
                                                                                 TEST_SIZE])

In [22]:
# pipe data through a dataloader for batching
BATCH_SIZE = 20
dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [23]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=3, padding=1), 
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2), 
            nn.Conv2d(8, 16, 3, stride=2, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2) ,
            nn.Conv2d(16, 64, 3, stride=2, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(64, 128, 3, stride=2, padding=1) 
            #nn.ReLU(True)
            #nn.MaxPool2d(2, stride=2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 3, stride=2), 
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 16, 3, stride=2, padding=1), 
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 3, stride=2, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 3, stride=3, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [26]:
# model = autoencoder().cuda()
NUM_EPOCHS = 10
LEARNING_RATE = 1e-3
model = autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

In [24]:
for data in dataloader:
    temp = data
    break

In [25]:
temp.shape

torch.Size([20, 1, 40, 173])

In [27]:
for epoch in range(NUM_EPOCHS):
    for batch in dataloader:
        # ===================forward=====================
        output = model(batch)
        loss = criterion(output, batch)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.data.item())) # EDIT: changed [0] to .item()

torch.save(model.state_dict(), './model_state.pth')

RuntimeError: Given input size: (64x1x4). Calculated output size: (64x0x2). Output size is too small